### Simple CNN with Pytorch

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
import subprocess
print(subprocess.getoutput('nvidia-smi'))

In [ ]:
!cat /etc/*-release

In [ ]:
!pip install gputil
!pip install psutil
!pip install humanize

In [ ]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()

In [ ]:
import os
import shutil
import numpy as np 
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torchvision
from torch import nn
import torch.optim as optim
from torch.utils import data
from torchvision import transforms
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/gdrive', force_remount=True)
# root_path = 'gdrive/My Drive/Colab Notebooks/Classif_img_CNN/datas/'
root_path = 'gdrive/My Drive/Formation simplon/Brief_image_classif_CNN/datas/'

In [ ]:
file = ''.join(root_path+'AnimalFace.zip')
file

In [ ]:
!mkdir datas
import zipfile
with zipfile.ZipFile(file,"r") as zip_ref:
    zip_ref.extractall("datas")

In [ ]:
IMG_SIZE = 128

In [ ]:
# train folders
os.mkdir('/content/datas/train/')
for img_class in os.listdir("/content/datas/Image"):
    os.mkdir('/content/datas/train/' + img_class[:-4] + '/')

In [ ]:
# test folders
os.mkdir('/content/datas/test/')
for img_class in os.listdir("/content/datas/Image"):
    os.mkdir('/content/datas/test/' + img_class[:-4] + '/')

In [ ]:
# form train dataset
for img_class in tqdm(os.listdir('/content/datas/Image/')):
    img_ls = os.listdir('/content/datas/Image/' + img_class)
    for img in img_ls[:int(len(img_ls) * 0.8)]:
        shutil.copy('/content/datas/Image/' + img_class + '/' + img, 
                    '/content/datas/train/' + img_class[:-4] + '/' + img)

In [ ]:
# form test dataset
for img_class in tqdm(os.listdir('/content/datas/Image/')):
    img_ls = os.listdir('/content/datas/Image/' + img_class)
    for img in img_ls[int(len(img_ls) * 0.8):]:
        shutil.copy('/content/datas/Image/' + img_class + '/' + img, 
                    '/content/datas/test/' + img_class[:-4] + '/' + img)

In [ ]:
train_data_path = "/content/datas/train/"
test_data_path = "/content/datas/test/"

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
    ])

# this function get folder with images
train_data = torchvision.datasets.ImageFolder(root=train_data_path,
                                              transform=transform)

test_data = torchvision.datasets.ImageFolder(root=test_data_path,
                                             transform=transform)

In [ ]:
batch_size=64
train_data_loader = data.DataLoader(train_data, shuffle=True,
                                    batch_size=batch_size)

test_data_loader  = data.DataLoader(test_data, shuffle=True, 
                                    batch_size=batch_size)

In [ ]:
class_names = train_data.classes

## Create the model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module): 
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(20*30*30, 1024)
        self.fc2 = nn.Linear(1024, len(class_names))

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        #print("STEP 1 :", x.shape)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #print("STEP 2 :", x.shape)
        x = x.view(x.shape[0],-1)
        #print("STEP 3 :", x.shape)
        x = F.relu(self.fc1(x))
        #print("STEP 4 :", x.shape)
        x = F.dropout(x, training=self.training)
        #print("STEP 5 : ", x.shape)
        x = self.fc2(x)
        return x

## Optimizer 

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"device : {device} (gpu)")
else:
    device = torch.device("cpu")
    print(f"device : {device}")

In [ ]:
learning_rate = 0.001

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
print(model)

## entreinement

In [ ]:
model.to(device)
# add optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, amsgrad=True)
epochs = 50
loss_fn = torch.nn.CrossEntropyLoss()
loss_lst, loss_val_lst = [], []

for epoch in range(epochs):
    training_loss = 0.0
    valid_loss = 0.0
    model.train()
    for batch in train_data_loader:
        optimizer.zero_grad()
        inputs, target = batch
        inputs = inputs.to(device)
        target = target.to(device)
        # ## debugg
        # model.forward(inputs)
        output = model(inputs)
        
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        training_loss += loss.data.item()
    loss_lst.append(training_loss)

    model.eval()
    num_correct = 0
    num_examples = 0
    for batch in test_data_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        output = model(inputs)
        targets = targets.to(device)
        loss = loss_fn(output,targets)
        valid_loss += loss.data.item()
        correct = torch.eq(torch.max(F.softmax(output), dim=1)[1],
                        targets).view(-1)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
    loss_val_lst.append(valid_loss)

    print('Epoch: {}, Training Loss: {:.2f}, \
        Validation Loss: {:.2f}, \
        accuracy = {:.2f}'.format(epoch, training_loss, \
        valid_loss, num_correct / num_examples))

In [ ]:
plt.figure(figsize=[12, 8])
plt.plot(range(len(loss_lst)), loss_lst, label='Training loss')
plt.plot(range(len(loss_val_lst)), loss_val_lst, label='Validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(frameon=False)

In [ ]:
# test-the-model
model.eval()  # it-disables-dropout
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Specify a path to save to
PATH = "/content/gdrive/MyDrive/Formation simplon/Brief_image_classif_CNN/CNN_simple_pytorch.pt"

# Save
torch.save(model.state_dict(), PATH)

### Confusion matrix

In [ ]:
all_output = np.array([])
all_targets = np.array([])

for batch in tqdm(test_data_loader):
    inputs, targets = batch
    inputs = inputs.to(device)
    output = model(inputs)
    all_output = np.concatenate([all_output, output.max(dim=1).indices.cpu().numpy()])
    all_targets = np.concatenate([all_targets, targets.numpy()])

In [ ]:
# plot confusion matrix
conf_matr = confusion_matrix(all_output, 
                             all_targets)
import itertools
def plot_confusion_matrix(cm, labels,
                          normalize=True,
                          title='Confusion Matrix (Validation Set)',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        #print('Confusion matrix, without normalization')
        pass

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.savefig('./figures/conf_matrix.png')

plt.figure(figsize=(20,10))
plot_confusion_matrix(conf_matr, labels=class_names, cmap='viridis')

In [ ]:
from skimage.transform import resize
from google.colab import files
image = files.upload()
print(image.keys())
for k, v in image.items(): pass

In [ ]:
from skimage.io import imread
from skimage.transform import resize
im = imread(k) # 333636a798.jpeg ## fda9a545f5.jpeg
im = resize(im, (IMG_SIZE, IMG_SIZE))
plt.imshow(im)
plt.axis('off')

In [ ]:
from torch.autograd import Variable
from PIL import Image

# An instance of your model.
img_pil = Image.open(k)
#img_pil.show()
img_tensor = transform(img_pil).float()
img_tensor = img_tensor.unsqueeze(0)
img_tensor = img_tensor.cuda()

fc_out = model(Variable(img_tensor))

output = fc_out.cpu().detach().numpy()
print(class_names[output.argmax()])